### Theano example implementation for data classification

This example should illustrate the functionality of Theano on the TensorFlow examples. It is structured similar to the other demos. The first block is concerned with data generation and the latter with two implementations in Theano: a single perceptron and a multilayer perceptron.

In [ ]:
# Importing all the things
%matplotlib notebook

import theano
import numpy as np
import numpy.random as rnd
import theano.tensor as T
import matplotlib.pyplot as plt

from numpy import linalg as la
from numpy.random import randn, randint
from playground import viz
from playground import DataGenerator

In [ ]:
datasets = DataGenerator()

fig = plt.figure('Available Datasets')
for i, name in enumerate(['circle', 'xor', 'gauss', 'spiral']):
    data, labels = getattr(datasets, name)(200)
    axis = plt.subplot(221+i)
    axis.set_title(name)
    axis.scatter(*zip(*data), c=labels, cmap='bwr')
fig.canvas.draw()

The following setting helps with debugging and proved to be rather useful - although you need to learn the error messages and what they are supposed to say as well.

In [ ]:
# debug settings (not that it would help a lot...)
theano.config.exception_verbosity='high'

The first implementation here shows a simple perceptron implementation in Theano. Take a look at the code and play around with the parameters for the number of training steps or change the loss function and see what happens.

In [ ]:
## LINEAR REGRESSION

N = 400 # number of samples
feats = 20 # dimensionality of features
D = (randn(N, feats), randint(size=N, low=0, high=2)) # rand data
training_steps = 10000

x = T.matrix("x")
y = T.vector("y")
w = theano.shared(randn(20), name="w")
b = theano.shared(0., name="b")

print("-------- Perceptron Model --------")
print(w.get_value(), b.get_value())

p_1 = 1 / (1 + T.exp(-T.dot(x, w)-b)) # probability that target = 1
prediction = p_1 > 0.5 # the prediction threshold
xent = -y*T.log(p_1) - (1-y)*T.log(1-p_1) # cross-entropy loss func
cost = xent.mean() + 0.01 * (w**2).sum() # the cost to minimize
gw, gb = T.grad(cost, [w, b])

train = theano.function(inputs = [x, y], outputs = [prediction, xent],
                        updates = {w : w-0.1*gw, b : b-0.1*gb})

predict = theano.function(inputs = [x], outputs = prediction)


for i in range(training_steps):
    pred, err = train(D[0], D[1])
    
print("-------- Training result -------")
print(w.get_value(), b.get_value())
print("Target values for the data: ")
print(D[1])
print("Predictions on the data: ")
print(predict(D[0]))

Now let's extend it to the MLP with a hidden layer. You may try to add further layers or change again the parameters of the problem. If you increase the epochs take also care of the debuging print statements (otherwise your computation slows down and the notebook can get stuck).  

In [ ]:
## MULTILAYER-PERCEPTRON

# Choose your problem!
# datasets.circle(), datasets.xor(), datasets.gauss(), datasets.spiral()
n = 1000
data, labels = datasets.xor(n)
D = (data, labels.flatten())

x = T.matrix("x")
y = T.vector("y")
w_1 = theano.shared(rnd.randn(2,10), name="w1")
b_1 = theano.shared(np.zeros((10,)), name="b1")
w_2 = theano.shared(rnd.randn(10), name="w2")
b_2 = theano.shared(0., name="b2")

print("-------- Multilayer Model --------")
print(w_1.get_value(), b_1.get_value())
print(w_2.get_value(), b_2.get_value())

# expression graph for forward propagation
out = T.nnet.sigmoid(-T.dot(T.nnet.sigmoid(-T.dot(x,w_1)-b_1), w_2)-b_2)

prediction = out > 0.5

# cross entropy as loss function
cross_ent = -y * T.log(out) - (1-y) * T.log(1-out)
# cost function
cost = cross_ent.mean() + 0.01 * (w_2**2).sum()
gw_1, gb_1, gw_2, gb_2 = T.grad(cost, [w_1, b_1, w_2, b_2])

# compile the model
train = theano.function(inputs = [x, y], outputs = [prediction, cross_ent],
                        updates = [(w_1, w_1-0.1*gw_1), (b_1, b_1-0.1*gb_1),
                                   (w_2, w_2-0.1*gw_2), (b_2, b_2-0.1*gb_2)])

predict = theano.function([x], prediction)

epochs = 1000
batch_size = 10
#fig = plt.figure('Intermediate Results')
for epoch in range(epochs):
    # get batches of training data
    b_indx = rnd.choice(n, batch_size)
    b_data = D[0][b_indx]
    b_target = D[1][b_indx]
    pred, err = train(D[0], D[1])
    
    if (epoch+1)%100 == 0:
        perf = 1 - sum(abs(predict(D[0]) - D[1])) / n
        #viz(D[0], predict(D[0], (epoch+1)/epochs, perf, fig, discretize=False))
        print("Training... {:3.2%} done with accuracy {:2.2f}".format((i+1)/epochs, perf))
    

print("-------- Training result -------")
print(w_1.get_value(), b_1.get_value())
print(w_2.get_value(), b_2.get_value())

print("Target values for the data: ")
print(D[1])
print("Predictions on the data: ")
print(predict(D[0]))